#### 参数搜索与sklearn实现
##### 参数类型
    - 一般参数：模型通过最小化损失函数自动求解的参数
    - 超参数：不能通过模型对数据进行学习而求解的参数，比如神经网络的层数、正则系数的alpha值等

##### 参数搜索
    -超参数的搜索：提前设置好参数可以选择的候选值，然后根据不同参数组合对于模型泛化能力的贡献，选取最佳的超参数组合

##### 参数搜索的方法：
GridSearchCV，基于交叉验证的网格搜索法：将要搜索的参数候选值输入搜索器内，搜索器遍历每一种参数组合，使用交叉验证法对比每种参数组合下模型的表现，返回表现最好模型的参数值
    - 优点：自动调参，参数准确性高
    - 缺点：需要耗费巨大的算例和计算时间（比如：搜索100棵树的随机森林模型的两种参数各三个候选值，选择k等于10的交叉验证，则需要训练验证9000棵决策树才能返回最佳参数）

RandomizedSearchCV,基于交叉验证的随机搜索法，基本原理与GridSearchCV一致，但为了提高搜索效率，搜索器会从参数组合中随机搜索一些参数进行训练和验证，返回其中表现最好的参数值
    - 优点：运行效率高，适合大数据样本
    - 缺点：参数的准确性有所牺牲


